In [1]:
import simpleFoam_tools as sft
import pandas as pd
import numpy as np

In [2]:
remove = 1
if remove:
    sft.remove_run_files(".")

In [3]:
domain_name = "channel1"  # "cylinder" or "vti_bent_x"

scale = 1e-6  # voxel size [m]
dp = 1.0           # pressure difference [Pa]
refinement = 0    # local refinement level
factor_mesh_x = 0.5    # global mesh scale factor
factor_mesh_y = 4    # global mesh scale factor
factor_mesh_z = 4    # global mesh scale factor

boundary_type = "Wall"  # "symmetryPlane" or "Wall"

dt = 1e-6
end_time = 50e-6
write_interval = 10

In [4]:
# Get voxel shape and a pore voxel location from VTI
vti_path = f"constant/geometry/{domain_name}.vti"
shape = sft.vti_shape(vti_path)
location_in_mesh = sft.find_pore_location(vti_path)

# Convert VTI to STL surface mesh
stl = f"{domain_name}.stl"
stl_path = f"constant/triSurface/{stl}"
sft.vti_to_stl(vti_path, stl_path)

# Adjust mesh resolution
mesh_resolution = (int(shape[0]*factor_mesh_x), int(shape[1]*factor_mesh_y), int(shape[2]*factor_mesh_z))

# File paths
blockMeshDict_path    = "system/blockMeshDict"
controlDict_path      = "system/controlDict"
p_field_path          = "0/p"
U_field_path          = "0/U"
snappyHexMeshDict_path = "system/snappyHexMeshDict"

# Generate OpenFOAM files
sft.generate_blockMeshDict(shape, mesh_resolution, blockMeshDict_path, boundary=boundary_type)
sft.generate_snappyHexMeshDict(location_in_mesh, stl, snappyHexMeshDict_path, refinement=refinement)
sft.generate_controlDict(controlDict_path, end_time=end_time, write_interval=write_interval, dt=dt)
sft.generate_pressure_field(p_field_path, dp=dp, boundary=boundary_type)
sft.generate_velocity_field(U_field_path, boundary=boundary_type)

Running command: pvpython /home/h09435ap/porePermFoam/simpleFoam_tools/paraview_stl.py /home/h09435ap/porePermFoam/constant/geometry/channel1.vti /home/h09435ap/porePermFoam/constant/triSurface/channel1.stl
hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
✅ Wrote ASCII STL: /home/h09435ap/porePermFoam/constant/triSurface/channel1.stl
Generated blockMeshDict at: system/blockMeshDict
Generated snappyHexMeshDict at: system/snappyHexMeshDict
Generated controlDict at: system/controlDict
Generated p at: 0/p with boundary zeroGradient
Generated U at: 0/U with boundary noSlip


In [5]:
sft.run_simplefoam(".", scale=scale)


>>> blockMesh

/*---------------------------------------------------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2412                                  |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
Build  : _f00230df-20250528 OPENFOAM=2412 patch=250528 version=2412
Arch   : "LSB;label=32;scalar=64"
Exec   : blockMesh
Date   : Aug 29 2025
Time   : 13:24:40
Host   : e-10aux32876g5
PID    : 518057
I/O    : uncollated
Case   : /home/h09435ap/porePermFoam
nProcs : 1
trapFpe: Floating point exception trapping enabled (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (fileM

In [6]:
df_q_area = pd.read_csv("q_in.csv")
df_q_area.head()

,time,flowRate_phi,area
0,0.00000,-1.053460e-15,1.185920e-10
1,0.00001,-1.833330e-15,1.185920e-10
2,0.00002,-2.440980e-15,1.185920e-10
3,0.00003,-2.926140e-15,1.185920e-10
4,0.00004,-3.317250e-15,1.185920e-10


In [7]:
phi = sft.vti_phi(vti_path)
print(f"Porosity: {phi:.02%}")

# Cross-sectional area
A = shape[1] * shape[2] * scale**2
miu = 1e-3  # fluid viscosity [Pa·s]
L = shape[0] * scale  # length [m]

# Flow rate from last timestep
Q = np.abs(df_q_area["flowRate_phi"].iloc[-1])

# Darcy's law
k = Q * miu * L / (A * dp)

print(f"Permeability: {k:.02e} m^2")
print(f"Permeability: {k * 1.01324997e15:.02f} md")

Porosity: 10.60%
Permeability: 3.32e-14 m^2
Permeability: 33.61 md
